# Baseline model

*Du passé faisons table rase*

We use as a simple baseline a model where only the information a time T is used.

### Import 

In [1]:
from sepsis.io import get_train, get_valid
from sepsis import constants

In [2]:
train, valid = get_train(), get_valid()

In [3]:
vars = constants.TRAIN_FEATURES
X_train, X_valid = train[vars], valid[vars]
y_train, y_valid = train['SepsisLabel'], valid['SepsisLabel']

### Imputation

Forward propagation for ease of use with groupby 

But interpolation should be tried out

In [4]:
X_train = X_train.groupby('ID').ffill()
X_valid =  X_valid.groupby('ID').ffill()

### Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import numpy as np
# need to design better imputation
pipe = Pipeline([('impute', SimpleImputer(missing_values=np.nan)),('scaler', StandardScaler()), ('lr', LogisticRegression(class_weight='balanced', C=0.5))])

In [10]:
from sklearn.model_selection import GridSearchCV
# should use custom challenge metrics 
# but too time consumming to adapt the code from the github  into a custom metrics for the scoring parameter (it does accept custom callback) 
# besides the challenge code is not vectorized and would take ages to evaluate
param_grid = [
  {'lr__C': [0.1, 0.5, 1, 10, 100, 1000]}
 ]
gs = GridSearchCV(pipe, param_grid, scoring='f1')

In [11]:
gs.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('impute', SimpleImputer()),
                                       ('scaler', StandardScaler()),
                                       ('lr',
                                        LogisticRegression(C=0.5,
                                                           class_weight='balanced'))]),
             param_grid=[{'lr__C': [0.1, 0.5, 1, 10, 100, 1000]}],
             scoring='f1')

In [ ]:
gs.best_score_

0.05534478677446324

not good ...

In [ ]:
from sklearn.model_selection import cross_validate
best = gs.best_estimator_
cross_validate(best, X_train, y_train, scoring=['accuracy', 'f1', 'recall', 'precision'])

{'fit_time': array([2.56453085, 2.64168382, 2.2443347 , 2.12499452, 2.26426649]),
 'score_time': array([0.41758394, 0.48809052, 0.33798933, 0.37431622, 0.33893943]),
 'test_accuracy': array([0.55792183, 0.55432072, 0.62294316, 0.70410099, 0.71147641]),
 'test_f1': array([0.05035953, 0.05239763, 0.05559087, 0.06171846, 0.05665745]),
 'test_recall': array([0.65188172, 0.68525986, 0.6171595 , 0.54121864, 0.48174692]),
 'test_precision': array([0.02619144, 0.02724026, 0.02910631, 0.03272515, 0.03009865])}

In [ ]:
# evaluate on valid but need to use custom metrics